In [44]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup

In [45]:
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get('https://www.youtube.com/user/jacksfilms/videos')


for _ in range(56):
  driver.find_element_by_tag_name('body').send_keys(Keys.END)
  time.sleep(0.5)

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

videos = soup.find_all('div',{'id':'dismissible'})

In [46]:

master_list = []

for video in videos:
  data_dict = {}
  data_dict['title'] = video.find('a',{'id':'video-title'}).text
  data_dict['video_url'] = 'https://www.youtube.com/' + video.find('a',{'id':'video-title'})['href']
  meta = video.find('div',{'id':'metadata-line'}).find_all('span')
  data_dict['views'] = meta[0].text
  data_dict['video_age'] = meta[1].text
  master_list.append(data_dict)


In [47]:
import pandas as pd

In [48]:
youtube_df = pd.DataFrame(master_list)

In [49]:
def convert_views(df):
  if 'K' in df['views']:
    views = float(df['views'].split('K')[0])*1000
    return views
  elif 'M' in df['views']:
    views = float(df['views'].split('M')[0])*1000000
    return views

In [50]:
youtube_df['CLEAN_VIEWS'] = youtube_df.apply(convert_views,axis=1)
youtube_df['CLEAN_VIEWS'] = youtube_df['CLEAN_VIEWS'].astype(int)

In [51]:
youtube_df.to_csv('output_youtube.csv',index=False)